# TMCA Water Usage

In [ ]:
# imports
import pandas as pd
import re
import plotly.express as px
import pingouin as pg

# display options
pd.options.display.max_columns = 100

# number of residents per unit mapping taken from https://www.tuckermathews.com/p/Residents
n_resident_dict = {
    'unit_1': 1,
    'unit_2': 2,
    'unit_3': 1,
    'unit_4': 1,
    'unit_5': 1,
    'unit_6': 1,
    'unit_7': 1,
    'unit_8': 2,
    'unit_9': 1,
    'unit_1r': 1,
    'unit_2r': 1,
    'unit_3r': 1,
    'unit_4r': 1
}

# convert to dataframe for joining purposes
n_resident_lkup = pd.DataFrame(
    n_resident_dict.items(), 
    columns=["unit", "n_residents"]
)

# read in trustee data, reshape, and rename columns
df = pd.read_excel(
    "/Users/grahamthomson/Downloads/TMCA Water Meter Readings .xlsx", 
    engine="openpyxl"
).transpose().reset_index()

units = ['unit_1', 'unit_2', 'unit_3',
     'unit_4', 'unit_5', 'unit_6',
     'unit_7', 'unit_8', 'unit_9',
     'unit_1r', 'unit_2r', 'unit_3r',
     'unit_4r']

df.columns = ["date"] + units

# filter out any row that isnt just a date and reading per unit
df = df[df["date"].apply(lambda x: True if re.match("\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", str(x)) else False)]

for c in df.columns:
    # create lagged readings
    df[f"{c}_previous_reading"] = df[f"{c}"].shift(1)
    
    # calculate reading difference and date diff
    if c == "date":
        df["days_between_readings"] = (df["date"] - df[f"{c}_previous_reading"])
        df["days_between_readings"] = df["days_between_readings"].apply(lambda x: x.days)
    else:
        df[f"{c}_difference"] = df[f"{c}"] - df[f"{c}_previous_reading"]
        df[f"{c}_avg_daily_difference"] = df[f"{c}_difference"] / df["days_between_readings"]

# sort columns
df = df[sorted(df.columns)]

In [ ]:
df[["date"] + [c for c in df.columns if c.startswith("unit_8")]]

## Total Usage by Unit

In [ ]:
total_usage = pd.DataFrame([[u, df[u].max() - df[u].min(), n_resident_dict[u]] for u in units], columns=["unit", "total_usage", "n_residents"])
total_usage["n_residents_percent"] = total_usage["n_residents"] / total_usage["n_residents"].sum()
total_usage["total_usage_percent"] = total_usage["total_usage"] / total_usage["total_usage"].sum()
total_usage["usage_index"] = total_usage["total_usage_percent"] / total_usage["n_residents_percent"]
total_usage = total_usage.sort_values("usage_index", ascending=False)

total_usage.style\
.background_gradient(subset=["total_usage_percent", "n_residents_percent", "usage_index"])\
.format("{:.2%}", subset=["total_usage_percent", "n_residents_percent"])\
.hide_index()

## Box Plots / Normalization

In [ ]:
# daily average usage
dau = df[[c for c in df.columns if c.endswith("_avg_daily_difference")]].dropna().melt()
dau.columns = ["unit", "avg_usage"]
dau["unit"] = dau["unit"].apply(lambda x: x.replace("_avg_daily_difference", ""))

dau_normalized = dau.merge(n_resident_lkup, on="unit")
dau_normalized["avg_usage_normalized"] = dau_normalized["avg_usage"] / dau_normalized["n_residents"]
dau_normalized = dau_normalized.melt(
    id_vars="unit", 
    value_vars=["avg_usage", "avg_usage_normalized"], 
    var_name="metric_name",
    value_name="liters_used"
)
px.box(
    data_frame=dau_normalized, 
    x="unit", 
    y="liters_used", 
    color="metric_name", 
    title="Average Daily Water Usage by Unit"
)

## Statistical Tests

In [ ]:
pg.ttest(
    x=dau_normalized.query("(unit == 'unit_8') & (metric_name == 'avg_usage_normalized')")["liters_used"].astype(float),
    y=dau_normalized.query("(unit == 'unit_2') & (metric_name == 'avg_usage_normalized')")["liters_used"].astype(float)
)

In [ ]:
pg.ttest(
    x=dau_normalized.query("(unit == 'unit_8') & (metric_name == 'avg_usage_normalized')")["liters_used"].astype(float),
    y=dau_normalized.query("(unit == 'unit_7') & (metric_name == 'avg_usage_normalized')")["liters_used"].astype(float)
)

In [ ]:
df.to_excel("./TMCA_water_usage_reshape.xlsx", header=True)
dau_normalized.to_excel("./TMCA_water_usage_normalized.xlsx", header=True)